In [1]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 

import pandas as pd

from random import seed
from random import randrange, uniform

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.webdriver import Options
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [2]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--log-level=3")  # fatal
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--incognito")

In [ ]:
def get_linkedin_active_account():
    username = 'zeipf6@gmail.com'
    password = 'dextra#2021'
    return username, password

In [17]:
def auth_linkedin(browser):
    
    linkedin_login_url = 'https://www.linkedin.com/uas/login'
    
    try:
        print('Accessing: ', linkedin_login_url)
        browser.get(linkedin_login_url)
        time.sleep(uniform(1,2))
    
        username, password = get_linkedin_active_account()
    
        print('Sign up with: ', username)
        username_field = browser.find_element_by_id('username')
        username_field.send_keys(username)

        password_field = browser.find_element_by_id('password')
        password_field.send_keys(password)
    
        time.sleep(uniform(1,2))
        password_field.submit()
    
    except:
        print('ERROR: AUTH LINKEDIN FAILS')

In [4]:
def get_jobs_list(browser, retry: bool = True):    
    try:
        element = WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-pagination__pages--number')))
        soup = BeautifulSoup(browser.page_source, features='html.parser')
        time.sleep(random())
        li_list = soup.find_all('li',attrs={'data-occludable-entity-urn' :True}, recursive=True)
        return li_list
    except:
        time.sleep(uniform(1,2))
        if retry: return get_job_description_on(browser, False)
        else: return []

In [5]:
def get_job_description_on(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        job_description = soup.find(attrs={"class": "jobs-description-content__text"}).getText(separator=' ')
        job_description = re.sub(r'\ +', ' ', job_description.replace('\n', '')).strip()
        return job_description
    except:
        time.sleep(uniform(1,2))
        if retry: return get_job_description_on(browser, False)
        else: return 'NF'

In [6]:
def get_job_applicants(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        applicants = soup.find(attrs={'class': 'jobs-details-job-summary'}).find(text=re.compile('\d\ applicant')).replace('\n', '').strip()
        applicants = re.sub(r'[a-zA-Z\ ]', '', applicants)
        return applicants
    except:
        time.sleep(uniform(1,2))
        if retry: return get_job_applicants(browser, False)
        else: return 'NF'

In [7]:
def get_job_employees(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        employees = soup.find(attrs={'class': 'jobs-details-job-summary'}).find(text=re.compile('[\+\d]\ employees')).replace('\n', '').strip()
        employees = re.sub(r'[a-zA-Z\ ]', '', employees)
        return employees
    except:
        time.sleep(uniform(1,2))
        if retry: return get_job_employees(browser, False)
        else: return 'NF'

In [8]:
def get_job_views(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        views = soup.find(attrs={'class': 'jobs-details-top-card__content-container'}).find(text=re.compile('\d\ view')).replace('\n', '').strip()
        views = re.sub(r'[a-zA-Z\ ]', '', views)
        return views
    except:
        time.sleep(uniform(1,2))
        if retry: return get_job_views(browser, False)
        else: return 'NF'

In [9]:
def get_job_date(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        job_li = soup.find('li',{'data-occludable-entity-urn': 'urn:li:fs_normalized_jobPosting:' + job_number})
        job_date = job_li.find("time")["datetime"]
    except:
        time.sleep(uniform(1,2))
        if retry: return get_job_date(browser, False)
        else: return 'NF'

In [10]:
def find_company_title_on(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        company_title = soup.find(attrs={"class": "org-top-card-summary__title"})
        company_title = company_title.find("span").getText()
        return company_title
    except:
        time.sleep(uniform(1,2))
        if retry: return find_company_title_on(browser, False)
        else: return 'NF'

In [11]:
def find_company_employees_on(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        employees = soup.find(attrs={"class": "org-about-company-module__company-size-definition-text"})
        employees = employees.getText().replace('\n', '').strip()
        return employees
    except:
        time.sleep(uniform(1,2))
        if retry: return find_company_employees_on(browser, False)
        else: return 'NF'

In [12]:
def find_company_details_dict_on(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        company_details_dict = {}
        company_details = soup.find('dl', attrs={'class': 'overflow-hidden'}).find_all()
        for num, info in enumerate(company_details):
            if(info.name == 'dt'):
                company_details_dict[info.text.replace('\n', '').strip()] = company_details[num+1].text.replace('\n', '').strip()
        return company_details_dict
    except:
        time.sleep(uniform(1,2))
        if retry: return find_company_details_dict_on(browser, False)
        else: return {}

In [13]:
def find_total_jobs_on(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        total_jobs = re.search(r"(.+\ )", soup.find("div", attrs={"class": "jobs-search-results-list__title-heading"}).find("small").text)
        total_jobs = total_jobs.group(1).strip()
        return total_jobs
    except:
        time.sleep(uniform(1,2))
        if retry: return find_total_jobs_on(browser, False)
        else: return 'NF'

In [14]:
def find_usa_iteng_jobs_on(browser, retry: bool = True):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    try:
        iteng_jobs = re.search(r"(.+\ )", soup.find("div", attrs={"class": "jobs-search-results-list__title-heading"}).find("small").text)
        iteng_jobs = iteng_jobs.group(1).strip()
        return iteng_jobs
    except:
        time.sleep(uniform(1,2))
        if retry: return find_usa_iteng_jobs_on(browser, False)
        else: return 'NF'

In [18]:
browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
auth_linkedin(browser)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/home/carlos.cayres/.wdm/drivers/chromedriver/linux64/88.0.4324.96/chromedriver] found in cache


Accessing:  https://www.linkedin.com/uas/login
Sign up with:  zeipf6@gmail.com


In [16]:
print(find_company_title_on(browser))
print(find_company_employees_on(browser))
print(find_company_details_dict_on(browser))
print(find_total_jobs_on(browser))
print(find_usa_iteng_jobs_on(browser))

Apple
+ de 10.001 funcionários
{'Site': 'http://www.apple.com/jobs/', 'Setor': 'Produtos eletrônicos', 'Tamanho da empresa': '+ de 10.001 funcionários', 'Sede': 'Cupertino, California', 'Tipo': 'Empresa de capital aberto', 'Fundada em': '1976', 'Especializações': 'Innovative Product Development, World-Class Operations, Retail e Telephone Support'}
NF
NF


In [32]:
print(get_jobs_list(browser))
print(get_job_description_on(browser))
print(get_job_applicants(browser))
print(get_job_employees(browser))
print(get_job_views(browser))
print(get_job_date(browser))

NF
NF
NF
NF
NF
NF
